In [ ]:
"""
Created on Thu Jan 16 14:55 2023

Trying to do overview plots like Pierre

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf

In [2]:
sns.set_context('paper')

In [21]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [3]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/CDO_PROCESSED/'

In [4]:
T_grid_vars_closed = xr.open_dataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open = xr.open_dataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open = T_grid_vars_open.where(T_grid_vars_open['so'] > 0)

In [5]:
var_list = ['global_temp','wed_gyre','ross_gyre','ACC',
            'FRIS_melt','Ross_melt','LarsenC_melt','total_melt',
            'mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant',
            'salbot_WW','salbot_EW','salbot_WR','salbot_ER',
            'theta_WW','theta_EW','theta_WR','theta_ER']

In [17]:
var_obs_mean = xr.DataArray(data=np.array([15.0, 56.0, 20.0, 136.7, 
               155.4, 47.7, 20.7, 155.4 + 47.7 + 20.7,
               15.3, 6.3, 3.3, 19.8, 
               34.9, np.nan, 35.0, np.nan, 
               np.nan, -1.95, -1.9, -1.7]), dims='var').assign_coords({'var': var_list})

In [18]:
var_obs_std = xr.DataArray(data=np.array([0, 8.0, 5.0, 7.8, 
              45.0, 34.0, 67.0, 67.0,
              1.0, 1.0, 0.5, 0.5, 
              0.0, np.nan, 0.0, np.nan,
              np.nan, 0.2, 0.4, 0.4]), dims='var').assign_coords({'var': var_list})

In [24]:
u_open = xr.open_dataset(clara_path+'n42openc/n42openc_0-100_1Y_uo_vertsum.nc')

In [28]:
u_open['uocetr_eff'].isel(time_counter=0) # TO BE CONTINUED HERE!!!!! (think about how to define the region, ideally with x and y)

In [22]:
f = plt.figure()
f.set_size_inches(8.25*5, 8.25*5)

ax={}

leg_hdl = []

i = 0


for vv in var_list:
    
    ax[i] = f.add_subplot(5,5,i+1)

    if vv == 'global_temp':
        temp_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1Y_thetao_fldmean.nc').isel(deptht=0).squeeze()
        temp_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1Y_thetao_fldmean.nc').isel(deptht=0).squeeze()
        ax[i].plot(temp_open['thetao'], color='deepskyblue')
        ax[i].plot(temp_closed['thetao'], color='royalblue')

    elif vv == 'ACC':
        u_open = xr.open_dataset(clara_path+'n42openc/n42openc/n42openc_0-100_1Y_uo_vertsum.nc').isel(deptht=0).squeeze()
        u_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21/n42tm21_0-100_1Y_uo_vertsum.nc').isel(deptht=0).squeeze()
        ax[i].plot(temp_open['uocetr_eff'], color='deepskyblue')
        ax[i].plot(temp_closed['uocetr_eff'], color='royalblue')
    
    ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
    ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)

    ax[i].set_title(vv)

    i = i+1
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()


libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
